# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
#url="https://www.imdb.com/search/title/"
#when adding filters, url will change adding the filters in
#"https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=1993-01-01,2010-12-31&user_rating=2.1"
#since we are being asked for title type, SD and ED and user ratings, we should take the url with those parameters as variables in f string

In [8]:
def scrape_imdb(title_type, start_date, end_date, user_rating, n_pages):
  for i in range(1,n_pages+1):
    n_pages = 1 + 50*(i-1)
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},&start={n_pages}&ref_=adv_nxt"
    response = requests.get(url)
    soup = BeautifulSoup(response.content)

    film_dict = {}
    index = 0

    films = soup.find_all("div", attrs = {"class":"lister-item mode-advanced"})
    for film in films:
      #getting the position, name and year
      film_header = film.find_all("h3", attrs = {"class":"lister-item-header"})
      for item in film_header:
        film_position = item.find_all("span", attrs = {"class":"lister-item-index unbold text-primary"})[0].get_text().strip()
        film_name = item.find_all("a")[0].get_text().strip()
        film_year = item.find_all("span", attrs = {"class":"lister-item-year text-muted unbold"})[0].get_text().strip()

      #getting more film data
      film_genre = film.find_all("span", attrs = {"class":"genre"})[0].get_text().strip()
      film_duration = film.find_all("span", attrs = {"class":"runtime"})[0].get_text().strip()
      film_rating = film.find_all("div", attrs = {"class":"inline-block ratings-imdb-rating"})[0].get_text().strip()
      film_description = film.find_all("p", attrs = {"class":"text-muted"})[1].get_text().strip()

      film_director_stars = film.find_all("p")[2] #currently this will return not only the director but also the actors!
      film_director = film_director_stars.find_all("a")[0].get_text().strip() #assuming the first position contains data regarding the director
      film_actors = ",".join([actor.get_text().strip() for actor in film_director_stars.find_all("a")[1:]]) #assuming from position 1 forward we have actors



      film_dict[index] = {"ranking":film_position,
                          "title":film_name,
                          "year":film_year,
                          "genre":film_genre,
                          "duration":film_duration,
                          "rating":film_rating,
                          "description": film_description,
                          "director":film_director,
                          "actors":film_actors
                        }
      index +=1
    return pd.DataFrame.from_dict(film_dict, orient="index")

scrape_imdb(title_type="feature", start_date="1990-01-01", end_date="1992-12-31", user_rating="7.5",  n_pages = int(input("how many pages you want?")))

,ranking,title,year,genre,duration,rating,description,director,actors
0,1.,O Silêncio dos Inocentes,(1991),"Crime, Drama, Thriller",118 min,8.6,A young F.B.I. cadet must receive the help of ...,Jonathan Demme,"Jodie Foster,Anthony Hopkins,Scott Glenn,Ted L..."
1,2.,Tudo Bons Rapazes,(1990),"Biography, Crime, Drama",145 min,8.7,The story of Henry Hill and his life in the ma...,Martin Scorsese,"Robert De Niro,Ray Liotta,Joe Pesci,Lorraine B..."
2,3.,Eduardo Mãos de Tesoura,(1990),"Drama, Fantasy, Romance",105 min,7.9,The solitary life of an artificial man - who w...,Tim Burton,"Johnny Depp,Winona Ryder,Dianne Wiest,Anthony ..."
3,4.,Exterminador Implacável 2 - O Dia do Julgamento,(1991),"Action, Sci-Fi",137 min,8.6,"A cyborg, identical to the one who failed to k...",James Cameron,"Arnold Schwarzenegger,Linda Hamilton,Edward Fu..."
4,5.,Sozinho em Casa,(1990),"Comedy, Family",103 min,7.7,"An eight-year-old troublemaker, mistakenly lef...",Chris Columbus,"Macaulay Culkin,Joe Pesci,Daniel Stern,John Heard"
5,6.,Danças com Lobos,(1990),"Adventure, Drama, Western",181 min,8.0,"Lieutenant John Dunbar, assigned to a remote w...",Kevin Costner,"Kevin Costner,Mary McDonnell,Graham Greene,Rod..."
6,7.,Misery - O Capítulo Final,(1990),"Drama, Thriller",107 min,7.8,After a famous author is rescued from a car cr...,Rob Reiner,"James Caan,Kathy Bates,Richard Farnsworth,Fran..."
7,8.,Cães Danados,(1992),"Crime, Thriller",99 min,8.3,When a simple jewelry heist goes horribly wron...,Quentin Tarantino,"Harvey Keitel,Tim Roth,Michael Madsen,Chris Penn"
8,9.,Uma Questão de Honra,(1992),"Drama, Thriller",138 min,7.7,Military lawyer Lieutenant Daniel Kaffee defen...,Rob Reiner,"Tom Cruise,Jack Nicholson,Demi Moore,Kevin Bacon"
9,10.,O Meu Primo Vinny,(1992),"Comedy, Crime",120 min,7.6,Two New Yorkers accused of murder in rural Ala...,Jonathan Lynn,"Joe Pesci,Marisa Tomei,Ralph Macchio,Mitchell ..."


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [9]:
# Your solution goes here